# Uncertainties support
As of now, basic operations are transparently handled.

Known issues : 
 - `x.nominal_value` will return the float nominal value, while you would like a quantity value (3 m, not 3). Same goes for `std_dev` and `std_score`
 - `uncertainties.umath` will fail on non-dimensionless objects, but that the case also for Quantity with physipy.math
 - there probably is a need for work on `(2*x*m+3*m).derivatives[x]` to be done
 
No array support testing has been done yet.
Also, some printing/formating testing must be done.

In [1]:
import numpy as np
import physipy
from physipy.quantity.utils import asqarray
from physipy import m, K, s, Quantity, Dimension

In [2]:
import uncertainties
from uncertainties import ufloat
from uncertainties import umath
from uncertainties.umath import *  # sin(), etc.

Define a quantity that hold the uncertainties value : 

In [3]:
height = ufloat(1.84, 0.1) 
qheight = height*m

print(height)
print(qheight)

1.84+/-0.10
1.84+/-0.10 m


Uncertainties attributes are still available but without unit (hence the need of a better interface):

In [4]:
print(qheight.nominal_value)
print(qheight.std_dev)
print(qheight.std_score(3))

1.84
0.1
11.599999999999998


Some operations fails like : 

In [66]:
u = ufloat(1, 0.1) #* m
v = ufloat(10, 0.1)# * m
sum_value = u + v
sum_value.derivatives[u]#]

1.0

## Operations with other quantities 
are possible as long as uncertainties support the operation with the quantity's value : 

In [6]:
print(qheight*2)
print(qheight*2*m)
print(qheight**2)
print(qheight*np.arange(3))
print((2*qheight+1*m))

3.68+/-0.20 m
3.68+/-0.20 m**2
3.4+/-0.4 m**2
[0.0+/-0 1.84+/-0.1 3.68+/-0.2] m
4.68+/-0.20 m


## Operations with other uncertainties

By default, an uncertainty that is not wrapped by a quantity is supposed to have no physical dimension

In [7]:
print(qheight)
print(qheight * ufloat(2, 0.1))
print(qheight / ufloat(2, 0.1))


1.84+/-0.10 m
3.68+/-0.27 m
0.92+/-0.07 m


## Access to the individual sources of uncertainty
based on https://pythonhosted.org/uncertainties/user_guide.html#access-to-the-individual-sources-of-uncertainty
Again, we loose the unit falling back on the backend value : we would like to have : 
```
21.00+/-0.22 m
v variable: 0.2 m
u variable: 0.1 m
```

In [8]:
u = ufloat(1, 0.1, "u variable") * m  # Tag
v = ufloat(10, 0.1, "v variable") * m
sum_value = u+2*v
print(sum_value)
for (var, error) in sum_value.error_components().items():
    print("{}: {}".format(var.tag, error))

21.00+/-0.22 m
v variable: 0.2
u variable: 0.1


# Comparison

In [9]:
x = ufloat(0.20, 0.01) *m
y = x + 0.0001*m

print(y > x) # expect True
print(y > 0*m) # expect True

y = ufloat(1, 0.1) * m
z = ufloat(1, 0.1) * m
print(y)
print(z)
print(y == y) # expect True
print(y == z) # expect False

True
True
1.00+/-0.10 m
1.00+/-0.10 m
True
False


# Math module and numpy
Not tested but will most likely fails as uncertainties relies on `umath` and `unumpy`. To be fair, physipy also have a `math` module that wraps the builtin one.

# Dirty Sandbox

In [80]:
import uncertainties as uc
x =  uc.ufloat(0.20, 0.01)
arr = np.array([uc.ufloat(1, 0.01),
                uc.ufloat(2, 0.1)])

arr_q = np.array([arr[0]*m, arr[1]*m])
arr.sum() * m == arr_q.sum()

True

In [77]:
np.isclose(arr_q.sum().value, (arr.sum() * m).value)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

3.0+/-0.1004987562112089

In [74]:
arr_q.sum().value == (arr.sum() * m).value

False

# Measurement : mix between Uncertainties and Pint
https://pint.readthedocs.io/en/stable/measurement.html?highlight=uncertainty

In [15]:
import numpy as np
book_length = (20. * m).plus_minus(2.)
print(book_length.value)
print(2 * book_length)

AttributeError: Neither Quantity object nor its value (20.0) has attribute 'plus_minus'